# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [6]:
# your code here
import zipfile

In [8]:
zip_file_path = 'Orders.zip'
extracted_dir = 'your-code'

In [9]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [10]:
#oops i cheated a bit and manually dragged it to the right file :P

orders = pd.read_csv('Orders.csv')

In [11]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [15]:
# your code here
#lets group by customers first
grouped_customers = orders.groupby('CustomerID')

In [18]:
customer_total = grouped_customers.amount_spent.sum()
print(customer_total)

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
           ...   
18280      180.60
18281       80.82
18282      178.05
18283     2094.88
18287     1837.28
Name: amount_spent, Length: 4339, dtype: float64


In [22]:
quantile_75 = customer_total.quantile(0.75)
quantile_95 = customer_total.quantile(0.95)

In [43]:
orders['group'] = 'plebs'

In [44]:
orders.loc[orders['amount_spent'] > quantile_95, 'group'] = 'VIP'

In [45]:
orders.loc[(orders['amount_spent'] > quantile_75) & (orders['amount_spent'] <= quantile_95), 'group'] = 'Preferred'

In [46]:
uniques = orders.group.value_counts()
print(uniques)

plebs        397792
Preferred       125
VIP               7
Name: group, dtype: int64


In [ ]:
#thats a lot of plebs lol

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [47]:
vips = orders[orders['group'] == 'VIP']

In [53]:
vip_country = vips.groupby('Country').size()
vip_country.head()

Country
United Kingdom    7
dtype: int64

In [ ]:
#so its the UK with the only and thus most vips. 

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [54]:
# your code here
# lets make a new group called Vippref, combining these two groups. (so quantile 0.75+)

orders2 = orders.copy()

In [55]:
orders2.loc[orders['amount_spent'] > quantile_75, 'group'] = 'Vippref'

In [57]:
vipprefs = orders[orders['group'] == 'Vippref']

In [58]:
vippref_country = vipprefs.groupby('Country').size()

In [60]:
vippref_country.head()
#idk why it didnt work? 

Series([], dtype: int64)